In [4]:
from external.risearch.RIsearch1.numpy_to_csv import dsm_variable_to_csv,numpy_to_csv
dsm_variable_to_csv()

In [5]:
import pytest

from asodesigner.fold import get_mfe_scores, get_trigger_mfe_scores_by_risearch, Interaction, dump_target_file
from asodesigner.target_finder import get_gfp_first_exp
from asodesigner.util import get_antisense


def test_empty():
    name_to_sequence = {"GFP": get_gfp_first_exp()}

    result = get_trigger_mfe_scores_by_risearch(name_to_sequence['GFP'][0:20], name_to_sequence, minimum_score=30000,
                                                # very high score, no candidates will be found
                                                parsing_type='2')
    mfe_scores = get_mfe_scores(result, '2')

    assert len(mfe_scores) == 1
    assert len(mfe_scores[0]) == 0



def test_risearch():
    # example
    name_to_sequence = {
        "0": "AUGUGUUCAAUUUUUGGCGUAUUCGAUAUCAAAACAGACGCAGUUGAGCUGCGUAAGAAAGCACUCGAGCUGUCACGCCUGAUGCGUCAUCGUGGCCCGGACUGGUCCGGUAUUUAUGCCAGCGAUAACGCCAUUCUCGCCCACGAACGUCUGUCAAUUGUUGACGUUAACGCGGGGGCGCAACCUCUCUACAACCAACAAAAAACUCAUGUGCUGGCGGUAAACGGUGAAAUCUACAACCACCAGGCACUGCGCGCCGAAUAUGGCGAUCGUUAUCAGUUCCAGACCGGAUCUGACUGUGAAGUGAUCCUCGCGCUGUAUCAGGAAAAAGGGCCGGAAUUUCUCGACGACUUGCAGGGCAUGUUUGCCUUUGCCUUGUACGACAGCGAAAAAGAUGCCUACCUGAUUGGUCGCGACCAUCUGGGGAUCAUCCCACUGUAUAUGGGCUAUGACGAACACGGUCAGCUGUAUGUGGCCUCAGAAAUGAAAGCCCUGGUGCCAGUUUGCCGCACGAUUAAAGAGUUCCCGGCGGGGAGCUAUUUGUGGAGCCAGGACGGCGAAAUCCGUUCUUAUUAUCAUCGCGACUGGUUCGACUACGAUGCGGUGAAAGAUAACGUAACCGACAAAAACGAGCUGCGUCAGGCACUUGAAGAUUCCGUUAAAAGCCAUCUGAUGUCUGAUGUGCCUUACGGUGUGCUGCUUUCUGGUGGUCUGGAUUCCUCAAUUAUUUCCGCUAUCACCAAGAAAUACGCAGCCCGUCGCGUGGAAGAUCAGGAACGCUCUGAAGCCUGGUGGCCGCAGUUACACUCCUUUGCUGUAGGUCUGCCGGGUUCACCGGAUCUUAAAGCAGCCCAGGAAGUGGCAAACCAUCUGGGCACGGUGCAUCACGAAAUUCACUUCACUGUACAGGAAGGUCUGGAUGCCAUCCGCGACGUGAUUUACCACAUCGAAACUUAUGAUGUGACCACAAUUCGCGCUUCAACACCGAUGUAUUUAAUGUCGCGUAAGAUCAAGGCGAUGGGCAUUAAAAUGGUGCUGUCCGGUGAAGGUUCUGAUGAAGUGUUUGGCGGUUAUCUUUACUUCCAUAAAGCGCCCAACGCUAAAGAACUGCAUGAAGAGACGGUGCGUAAACUGCUGGCCCUGCAUAUGUAUGACUGCGCGCGCGCCAACAAAGCGAUGUCAGCCUGGGGCGUGGAAGCACGCGUUCCGUUCCUCGACAAAAAAUUCCUUGAUGUGGCGAUGCGCAUUAACCCGCAGGAUAAAAUGUGCGGUAACGGCAAAAUGGAAAAACACAUCCUGCGUGAAUGUUUUGAGUCAUACCUGCCCGCAAGCGUGGCCUGGCGGCAGAAAGAGCAGUUCUCCGAUGGCGUCGGUUACAGUUGGAUCGACACCCUGAAAGAAGUGGCGGCGCAGCAGGUUUCUGAUCAGCAACUGGAAACUGCCCGCUUCCGCUUCCCGUACAACACGCCAACCUCAAAAGAAGCGUAUCUGUACCGGGAGAUCUUUGAAGAACUGUUCCCGCUUCCGAGCGCCGCUGAGUGCGUGCCUGGCGGUCCUUCCGUCGCGUGUUCUUCCGCUAAAGCGAUCGAAUGGGAUGAAGCGUUCAAGAAAAUGGACGAUCCAUCUGGUCGUGCGGUUGGUGUUCACCAGUCGGCAUAUAAGUAA",
        "1": "AUGUUCGAACAACGCGUAAAUUCUGACGUACUGACCGUUUCUACCGUUAACUCUCAGGAUCAGGUAACCCAAAAACCCCUGCGUGACUCGGUUAAACAGGCACUGAAGAACUAUUUUGCUCAACUGAAUGGUCAGGAUGUGAAUGACCUCUAUGAGCUGGUACUGGCUGAAGUAGAACAGCCCCUGUUGGACAUGGUGAUGCAAUACACCCGUGGUAACCAGACCCGUGCUGCGCUGAUGAUGGGCAUCAACCGUGGUACGCUGCGUAAAAAAUUGAAAAAAUACGGCAUGAACUAA"
    }

    target_path = dump_target_file('target-cache.fa', name_to_sequence)
    result = get_trigger_mfe_scores_by_risearch("UAGAUGCGCCACUUGUGGUAUUCCCGCAUC", name_to_sequence, minimum_score=900,
                                                parsing_type='2', target_file_cache=str(target_path))
    print(result)
    mfe_scores = get_mfe_scores(result, '2')

    print(mfe_scores)


def test_bad_fit():
    sense = 'TTTTTTTCTTCCATT'

    result = get_trigger_mfe_scores_by_risearch(sense, {'0': sense + sense}, parsing_type='2', minimum_score=900)
    print(result)

    gfp_seq = get_gfp_first_exp(gap=0)
    sample_seq = gfp_seq[0:15]
    result = get_trigger_mfe_scores_by_risearch('T' + sample_seq[1:15], {"gfp": gfp_seq}, parsing_type='2',
                                                minimum_score=900)
    print(result)


def test_risearch_gfp():
    gfp_seq = get_gfp_first_exp(gap=0)
    sample_seq = gfp_seq[:20]
    print("GFP ontarget(?) : ", gfp_seq[695:714])
    print("Sample", sample_seq)
    print("Sample antisense", get_antisense(sample_seq))
    # name_to_seq = {f"gfp_seq{i}" : gfp_seq for i in range(100)}
    name_to_seq = {f"gfp_seq": gfp_seq}
    result = get_trigger_mfe_scores_by_risearch(sample_seq, name_to_seq,
                                                interaction_type=Interaction.MODIFIED,
                                                minimum_score=900, neighborhood=30, parsing_type='2')
    print(result)

    mfe_scores = get_mfe_scores(result, '2')
    print(mfe_scores)

    bad_samples = [s + sample_seq[3:20] for s in ["AAA", "ATA", "AGA", "ACG"]]

    for bad_sample in bad_samples:
        result = get_trigger_mfe_scores_by_risearch(bad_sample, name_to_seq,
                                                    interaction_type=Interaction.MODIFIED, minimum_score=900,
                                                    neighborhood=30, parsing_type='2')
        print(result)

        mfe_scores = get_mfe_scores(result, '2')
        print(mfe_scores)


In [6]:
test_risearch_gfp()

GFP ontarget(?) :  CGGCATGGACGAGCTGTAC
Sample ATGGTGAGCAAGGGCGAGGA
Sample antisense TCCTCGCCCTTGCTCACCAT
trigger	1	20	gfp_seq	1	20	1929	-28.80
trigger	2	13	gfp_seq	95	106	1086	-15.57

[[-28.8, -15.57]]
trigger	1	17	gfp_seq	4	20	1744	-25.15
trigger	2	13	gfp_seq	95	106	1086	-15.57

[[-25.15, -15.57]]
trigger	1	17	gfp_seq	4	20	1744	-25.15
trigger	2	13	gfp_seq	95	106	1086	-15.57

[[-25.15, -15.57]]
trigger	1	17	gfp_seq	4	20	1744	-25.15
trigger	2	13	gfp_seq	95	106	1086	-15.57

[[-25.15, -15.57]]
trigger	1	18	gfp_seq	3	20	1894	-27.25
trigger	2	13	gfp_seq	95	106	1086	-15.57

[[-27.25, -15.57]]
